# Case 3 -- Steady Constant Sea Level (from Joe)



In [ ]:
import pathlib as pl
import numpy as np
import matplotlib.pyplot as plt
import flopy
import flopy.plot.styles as styles

# path to mf6 shared library
# executables based on mf6 feat-swi branch at: 
#   https://github.com/langevin-usgs/modflow6/tree/feat-swi-correction

# Put the name of the mf6 executable into mf6exe.txt,
# which is not under version control.
with open(pl.Path("./mf6exe.txt"), "r") as f:
    mf6exe = f.readline().strip()
print(f"using executable: {mf6exe}")

sim_ws = pl.Path("./temp/case3")

In [ ]:
name = "mf6"

In [ ]:
delr = delc = 50.
xlen = 2050.
ylen = delc
nlay, nrow, ncol = 1, int(ylen / delc), int(xlen / delr)
nlay, nrow, ncol

In [ ]:
shape3d = (nlay, nrow, ncol)
shape2d = (nrow, ncol)

In [ ]:
top = 10.
botm = -100.

In [ ]:
# newton_options = "newton"
newton_options = None

In [ ]:
if newton_options is None:
    linear_acceleration="cg"
else:
    linear_acceleration="bicgstab"

In [ ]:
sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=sim_ws, exe_name=mf6exe, memory_print_option="all")

In [ ]:
tdis = flopy.mf6.ModflowTdis(sim, time_units="days")

In [ ]:
ims = flopy.mf6.ModflowIms(sim, outer_dvclose=1e-5, inner_dvclose=1e-6, linear_acceleration=linear_acceleration, complexity="simple")

In [ ]:
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, newtonoptions=newton_options)

In [ ]:
nghb = 1

In [ ]:
top = np.full(shape2d, top, dtype=float)
top[0, :nghb] = 0
top

In [ ]:
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, top=top, botm=botm)

In [ ]:
ic = flopy.mf6.ModflowGwfic(gwf, strt=top)

In [ ]:
npf = flopy.mf6.ModflowGwfnpf(gwf, k=10, icelltype=1, alternative_cell_averaging="AMT-LMK",)

In [ ]:
zeta_file = name + '.zta'
swi = flopy.mf6.ModflowGwfswi(gwf, zeta_filerecord=zeta_file, zetastrt=botm)

In [ ]:
rch_mult = 0.5
rch_rate = 4e-4 * rch_mult
rch_rate = np.full(shape2d, rch_rate, dtype=float)
rch_rate[0, :nghb] = 0.
rch_rate

In [ ]:
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=rch_rate)

In [ ]:
ghb_spd = [(0, 0, i, 0.0, 62.5) for i in range(nghb)]
ghb_spd

In [ ]:
ghb = flopy.mf6.ModflowGwfghb(gwf, stress_period_data=ghb_spd)

In [ ]:
oc = flopy.mf6.ModflowGwfoc(gwf, head_filerecord=f"{name}.hds", saverecord=[("head", "all")])

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

### Plot the results

In [ ]:
hds = gwf.output.head().get_data()

In [ ]:
hds.max(), hds.max()*-40

In [ ]:
xsect = flopy.plot.PlotCrossSection(model=gwf, line={"row": 0})
pa = xsect.plot_array(hds, head=hds)

In [ ]:
#zeta = swiapi.zeta_last
fpth = sim_ws / f"{gwf.name}.zta"
zeta_all = flopy.utils.HeadFile(fpth, text="zeta").get_alldata()
zeta = zeta_all[-1].flatten()
x = gwf.modelgrid.xcellcenters.flatten()
pxs = flopy.plot.PlotCrossSection(gwf, line={"row":0})
ax = pxs.ax
if ncol < 50:
    pxs.plot_grid()
#pxs.plot_bc("ghb")
ax.plot(x, hds.flatten(), "k-")
ax.plot(x, zeta.flatten(), "k--")
# freshwater
ax.fill_between(x, hds.flatten(), zeta, color="cyan")
# saltwater
ax.fill_between(x, zeta, botm, color="red")

In [ ]:
zeta.min(), zeta.max()